# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
weather_df = pd.read_csv('weather_data.csv')
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,berezhany,35,UA,1584526563,37,49.45,24.93,53.60,2.24
1,hilo,90,US,1584526405,83,19.73,-155.09,73.99,12.75
2,yar-sale,0,RU,1584526565,85,66.83,70.83,18.82,14.00
3,abu samrah,99,SY,1584526566,65,35.30,37.18,49.98,27.18
4,cherskiy,100,RU,1584526567,96,68.75,161.30,-9.42,5.35


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

# Generate map
fig1 = gmaps.figure(map_type="HYBRID")

# Add locations and weights
locations = weather_df[["Lat", "Lng"]]
weights = weather_df["Humidity"]

# Create heatmap layer
heatmap_layer = gmaps.heatmap_layer(
    locations=locations, weights=weights,
    max_intensity=max(weather_df["Humidity"]), point_radius=5.0
)

# Add locations and weights to map
fig1.add_layer(heatmap_layer)
fig1


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create dataframe to hold only locations with ideal weather conditions

# Filter out temps only between 70-80 degrees F (not inclusive)
best_weather_cities = weather_df.loc[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70)]

# Remove cities with wind speed 10 mph or greater
best_weather_cities = best_weather_cities.loc[(best_weather_cities["Wind Speed"] < 10)]

# Remove cities with any cloudiness
best_weather_cities = best_weather_cities.loc[(best_weather_cities["Cloudiness"] == 0)]

best_weather_cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
27,buraydah,0,SA,1584526593,34,26.33,43.98,78.80,9.17
51,san patricio,0,MX,1584526620,83,19.22,-104.70,73.38,5.19
131,arraial do cabo,0,BR,1584526709,88,-22.97,-42.03,75.20,5.82
155,puerto ayora,0,EC,1584526736,89,-0.74,-90.35,73.99,1.99
156,lavumisa,0,SZ,1584526737,53,-27.31,31.89,78.60,9.26


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = best_weather_cities

hotel_df["Hotel Name"] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
27,buraydah,0,SA,1584526593,34,26.33,43.98,78.80,9.17,
51,san patricio,0,MX,1584526620,83,19.22,-104.70,73.38,5.19,
131,arraial do cabo,0,BR,1584526709,88,-22.97,-42.03,75.20,5.82,
155,puerto ayora,0,EC,1584526736,89,-0.74,-90.35,73.99,1.99,
156,lavumisa,0,SZ,1584526737,53,-27.31,31.89,78.60,9.26,
215,puerto escondido,0,MX,1584526806,69,15.85,-97.07,70.32,3.00,
253,luderitz,0,NaN,1584526849,50,-26.65,15.16,75.20,3.36,
268,coahuayana,0,MX,1584526865,51,18.73,-103.68,70.86,3.24,
431,guinguineo,0,SN,1584527048,57,14.27,-15.95,78.80,6.93,
536,puerto baquerizo moreno,0,EC,1584527165,74,-0.90,-89.60,78.06,5.86,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
                   
params = {
    "location": "",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

#Using Google Places API to find the first hotel for each city located within 5000 meters of coordinates

for index, row in hotel_df.iterrows():
    params["location"] = str(row['Lat']) + "," + str(row['Lng'])
    response = requests.get(base_url, params=params).json()
    results = response['results']
    hotel_df.loc[index,'Hotel Name'] = results[0]['name']
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
27,buraydah,0,SA,1584526593,34,26.33,43.98,78.80,9.17,Buraydah
51,san patricio,0,MX,1584526620,83,19.22,-104.70,73.38,5.19,San Patricio
131,arraial do cabo,0,BR,1584526709,88,-22.97,-42.03,75.20,5.82,Cabo Frio
155,puerto ayora,0,EC,1584526736,89,-0.74,-90.35,73.99,1.99,Puerto Ayora
156,lavumisa,0,SZ,1584526737,53,-27.31,31.89,78.60,9.26,Lavumisa


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
heatmap_layer = gmaps.heatmap_layer(list(zip(locations['Lat'],locations['Lng'])))
popup_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure()

# Adding heatmap layer to the base layer
fig.add_layer(heatmap_layer)

# Add marker layer on top of heat map
fig.add_layer(popup_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))